In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
from numpy import array, zeros, ravel, pad, dot, uint8, zeros_like, sort, multiply, divide, int8

In [3]:
def sobel_filter(image):
    kernel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    kernel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

    dst_x = np.abs(img_convolve(image, kernel_x))
    dst_y = np.abs(img_convolve(image, kernel_y))
    # modify the pix within [0, 255]
    dst_x = dst_x * 255/np.max(dst_x)
    dst_y = dst_y * 255/np.max(dst_y)

    dst_xy = np.sqrt((np.square(dst_x)) + (np.square(dst_y)))
    dst_xy = dst_xy * 255/np.max(dst_xy)
    dst = dst_xy.astype(np.uint8)

    theta = np.arctan2(dst_y, dst_x)
    return dst, theta

def im2col(image, block_size):
    rows, cols = image.shape
    dst_height = cols - block_size[1] + 1
    dst_width = rows - block_size[0] + 1
    image_array = zeros((dst_height * dst_width, block_size[1] * block_size[0]))
    row = 0
    for i in range(0, dst_height):
        for j in range(0, dst_width):
            window = ravel(image[i:i + block_size[0], j:j + block_size[1]])
            image_array[row, :] = window
            row += 1

    return image_array

def img_convolve(image, filter_kernel):
    height, width = image.shape[0], image.shape[1]
    k_size = filter_kernel.shape[0]
    pad_size = k_size//2
    # Pads image with the edge values of array.
    image_tmp = pad(image, pad_size, mode='edge')

    # im2col, turn the k_size*k_size pixels into a row and np.vstack all rows
    image_array = im2col(image_tmp, (k_size, k_size))

    #  turn the kernel into shape(k*k, 1)
    kernel_array = ravel(filter_kernel)
    # reshape and get the dst image
    dst = dot(image_array, kernel_array).reshape(height, width)
    return dst

In [4]:
PI = 180

def gen_gaussian_kernel(k_size, sigma):
    center = k_size // 2
    x, y = np.mgrid[0 - center:k_size - center, 0 - center:k_size - center]
    g = 1 / (2 * np.pi * sigma) * np.exp(-(np.square(x) + np.square(y)) / (2 * np.square(sigma)))
    return g

def gaussian_filter(image, k_size, sigma):
    height, width = image.shape[0], image.shape[1]
    # dst image height and width
    dst_height = height-k_size+1
    dst_width = width-k_size+1

    # im2col, turn the k_size*k_size pixels into a row and np.vstack all rows
    image_array = zeros((dst_height*dst_width, k_size*k_size))
    row = 0
    for i in range(0, dst_height):
        for j in range(0, dst_width):
            window = ravel(image[i:i + k_size, j:j + k_size])
            image_array[row, :] = window
            row += 1

    #  turn the kernel into shape(k*k, 1)
    gaussian_kernel = gen_gaussian_kernel(k_size, sigma)
    filter_array = ravel(gaussian_kernel)

    # reshape and get the dst image
    dst = dot(image_array, filter_array).reshape(dst_height, dst_width).astype(uint8)

    return dst

def canny(image, threshold_low=15, threshold_high=30, weak=128, strong=255):
    image_row, image_col = image.shape[0], image.shape[1]
    # gaussian_filter
    gaussian_out = img_convolve(image, gen_gaussian_kernel(9, sigma=1.4))
    # get the gradient and degree by sobel_filter
    sobel_grad, sobel_theta = sobel_filter(gaussian_out)
    gradient_direction = np.rad2deg(sobel_theta)
    gradient_direction += PI

    dst = np.zeros((image_row, image_col))

    """
    Non-maximum suppression. If the edge strength of the current pixel is the largest compared to the other pixels 
    in the mask with the same direction, the value will be preserved. Otherwise, the value will be suppressed. 
    """
    for row in range(1, image_row - 1):
        for col in range(1, image_col - 1):
            direction = gradient_direction[row, col]

            if (
                0 <= direction < 22.5
                    or 15 * PI / 8 <= direction <= 2 * PI
                    or 7 * PI / 8 <= direction <= 9 * PI / 8
            ):
                W = sobel_grad[row, col - 1]
                E = sobel_grad[row, col + 1]
                if sobel_grad[row, col] >= W and sobel_grad[row, col] >= E:
                    dst[row, col] = sobel_grad[row, col]

            elif (PI / 8 <= direction < 3 * PI / 8) or (9 * PI / 8 <= direction < 11 * PI / 8):
                SW = sobel_grad[row + 1, col - 1]
                NE = sobel_grad[row - 1, col + 1]
                if sobel_grad[row, col] >= SW and sobel_grad[row, col] >= NE:
                    dst[row, col] = sobel_grad[row, col]

            elif (3 * PI / 8 <= direction < 5 * PI / 8) or (11 * PI / 8 <= direction < 13 * PI / 8):
                N = sobel_grad[row - 1, col]
                S = sobel_grad[row + 1, col]
                if sobel_grad[row, col] >= N and sobel_grad[row, col] >= S:
                    dst[row, col] = sobel_grad[row, col]

            elif (5 * PI / 8 <= direction < 7 * PI / 8) or (13 * PI / 8 <= direction < 15 * PI / 8):
                NW = sobel_grad[row - 1, col - 1]
                SE = sobel_grad[row + 1, col + 1]
                if sobel_grad[row, col] >= NW and sobel_grad[row, col] >= SE:
                    dst[row, col] = sobel_grad[row, col]

            """
            High-Low threshold detection. If an edge pixelâ€™s gradient value is higher than the high threshold
            value, it is marked as a strong edge pixel. If an edge pixelâ€™s gradient value is smaller than the high
            threshold value and larger than the low threshold value, it is marked as a weak edge pixel. If an edge
            pixel's value is smaller than the low threshold value, it will be suppressed.
            """
            if dst[row, col] >= threshold_high:
                dst[row, col] = strong
            elif dst[row, col] <= threshold_low:
                dst[row, col] = 0
            else:
                dst[row, col] = weak

    """
    Edge tracking. Usually a weak edge pixel caused from true edges will be connected to a strong edge pixel while
    noise responses are unconnected. As long as there is one strong edge pixel that is involved in its 8-connected
    neighborhood, that weak edge point can be identified as one that should be preserved.
    """
    for row in range(1, image_row):
        for col in range(1, image_col):
            if dst[row, col] == weak:
                if 255 in (
                        dst[row, col + 1],
                        dst[row, col - 1],
                        dst[row - 1, col],
                        dst[row + 1, col],
                        dst[row - 1, col - 1],
                        dst[row + 1, col - 1],
                        dst[row - 1, col + 1],
                        dst[row + 1, col + 1],
                ):
                    dst[row, col] = strong
                else:
                    dst[row, col] = 0

    return dst

In [5]:
img = cv2.imread('Lenna_(test_image).png', 0)
canny_dst = canny(img)
cv2.imshow('Filtro de Canny', canny_dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
img = cv2.imread('Lenna_(test_image).png', 0)
sobel_grad, sobel_theta = sobel_filter(img)
cv2.imshow('Filtro de Sobel', sobel_grad)
cv2.imshow('Theta Sobel', sobel_theta)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
img = cv2.imread('Lenna_(test_image).png', 0)
gaussian3x3 = gaussian_filter(img, 3, sigma=1)
gaussian5x5 = gaussian_filter(img, 5, sigma=0.8)
cv2.imshow('Filtro gausiano con mascara de 3x3', gaussian3x3)
cv2.imshow('Filtro gausiano con mascara de 5x5', gaussian5x5)
cv2.waitKey(0)
cv2.destroyAllWindows()